# Toronto Dwellings Analysis Dashboard

In this notebook, we compile the visualizations from the previous analysis to create a Panel dashboard.

In [107]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [108]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [109]:
# Read the Mapbox API key
load_dotenv()
mapbox_api_key = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(mapbox_api_key)

# Import Data

In [110]:
# Import the CSVs to Pandas DataFrames
file_path = Path("./data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("./data/toronto_neighbourhoods_coordinates.csv")
neighbourhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, functions are declared to manipulate data and create plots.

### Global available data

In [111]:
# Getting the data from the top 10 expensive neighbourhoods
def top_ten_expensive_neighbourhoods(data):
    
    most_expensive_neighbourhoods = (
        data[['neighbourhood', 'average_house_value']]
        .groupby(['neighbourhood'])
        .mean()
        .sort_values(by='average_house_value', ascending=False)
    )

    return most_expensive_neighbourhoods[:10]


# Calculate the mean number of dwelling types units per year
def mean_dwelling_types_per_year(data):

    dwelling_types = data.drop(columns=data.iloc[:,9:])

    mean_dwelling_types_per_year = (
        dwelling_types
        .groupby([dwelling_types.index, 'neighbourhood'])
        .mean()
        .astype(int)
    )

    return mean_dwelling_types_per_year

# Calculate the average monthly shelter costs for owned and rented dwellings
def average_monthly_shelter_costs(data):

    average_monthly_shelter_costs_per_year = (
        data[['shelter_costs_owned', 'shelter_costs_rented']]
        .groupby([to_data.index])
        .mean()
        .round(2)
    )

    return average_monthly_shelter_costs_per_year


### Panel Visualization Functions

In [112]:
# Define Panel visualization functions
def neighbourhood_map(data):
    """Neighbourhood Map"""
    average_house_value = average_house_value_by_neighbourhood(data)

    scatter_plot = px.scatter_mapbox(
        average_house_value,
        lat='lat',
        lon='lon',
        color='average_house_value',
        size='average_house_value',
        color_continuous_scale='Viridis',
        size_max=11,
        zoom=9.2,
        mapbox_style='carto-positron',
        hover_data={
            'neighbourhood': True,
            'lat': False,
            'lon': False
        }
    )

    return scatter_plot
    

def create_bar_chart(data, title, xlabel, ylabel, color):
    """
    Create a barplot based in the data argument.
    """
    bar_chart = data.hvplot.bar(
        xlabel=xlabel, 
        ylabel=ylabel, 
        color=color, 
        title=title,
        height=500,
        rot=90
    )

    return bar_chart

def create_bar_chart_row_facet(data, title, xlabel, ylabel, color):
    """
    Create a barplot based in the data argument.
    """
    bar_chart_row_facet = px.bar(
        data,
        x=xlabel, 
        y=ylabel, 
        color=color, 
        title=title,
        height=1000,
        facet_row='Year'
    )

    return bar_chart_row_facet
    

def create_line_chart(data, title, xlabel, ylabel, color):
    """
    Create a line chart based in the data argument.
    """
    line_chart = data.hvplot.line(
        xlabel=xlabel, 
        ylabel=ylabel, 
        color=color, 
        title=title, 
        figsize=(15,5)
    )

    return line_chart
    

def average_house_value(data):
    """
    Average house values per year.
    """
    average_house_value_per_year = (
        data[['average_house_value']]
        .groupby([data.index])
        .mean()
        .round(2)
    )

    return average_house_value_per_year
    

def average_house_value_by_neighbourhood(data):
    """
    Average house values by neighbourhood.
    """
    neighbourhood_mean_values = (
        data
        .groupby(['neighbourhood'])
        .mean()
    )

    merged_housing_data = neighbourhood_locations.merge(
        neighbourhood_mean_values, 
        left_on='neighbourhood', 
        right_on='neighbourhood'
    )

    return merged_housing_data
    

def number_dwelling_types(data):
    """
    Number of dwelling types per year
    """
    dwelling_types_per_year = (
        data
        .groupby([data.index])
        .sum()
    )

    dwelling_types_per_year.drop(columns=[
            'average_house_value', 
            'shelter_costs_owned', 
            'shelter_costs_rented'
        ], 
        inplace=True
    )

    return dwelling_types_per_year
    

def average_house_value_snapshot(data):
    """
    Average house value for all Toronto's neighbourhoods per year.
    """
    average_house_value_snapshot = (
        average_house_value_by_neighbourhood(data)
    )

    average_house_value_snapshot.reset_index(inplace=True)

    average_house_value_snapshot.columns = [
        'Year', 
        'Neighbourhood', 
        'Avg. House Value'
    ]

    return average_house_value_snapshot
    

def top_most_expensive_neighbourhoods(data):
    """
    Top 10 most expensive neighbourhoods.
    """
    top_ten_2001 = data[data['year'] == 2001][:10]
    top_ten_2011 = data[data['year'] == 2011][:10]
    top_ten_2006 = data[data['year'] == 2006][:10]
    top_ten_2016 = data[data['year'] == 2016][:10]

    # Join the dataframes
    top_ten_expensive_neighbourhoods_per_year = pd.concat(
        [top_ten_2001, top_ten_2006, top_ten_2011, top_ten_2016], 
        join='inner', 
        axis='rows'
    )
    

def sunburts_cost_analysis(data):
    """
    Sunburst chart to conduct a costs analysis of most expensive
    neighbourhoods in Toronto per year.
    """
    sunburst_chart = px.sunburst(
        data, 
        path=['year', 'neighbourhood'], 
        values='average_house_value', 
        color='average_house_value', 
        color_continuous_scale='blues',
    )

    return sunburst_chart
    

## Panel Dashboard

Finally, all plots are combined into a single dashboard view using Panel.

### Create the Welcome view

In [113]:
# Create a Title for the Dashboard
dashboard_title = pn.Column(
    '## Real Estate Analysis of Toronto from 2001 to 2016'
)

# Define a welcome text
welcome_message = '''
Welcome to Toronto's Real Estate Analysis dashboard. In this dashboard 
you will find historical house values, dwelling types by neghbourhood and 
their associated costs over the years. Navigate through the tabs above to 
see more details on the evolution of Toronto's real estate market."
'''

# Create the Welcome view
scatter_plot = neighbourhood_map(to_data)
welcome_column = pn.Column(
    welcome_message,
    scatter_plot
)

### Create the Yearly Market Analysis view

In [114]:

# Get the mean number of dwelling types per year
dwelling_types_per_year = number_dwelling_types(to_data)

# Plot data for each year
bar_chart_2001 = create_bar_chart(
    dwelling_types_per_year.loc[2001], 
    'Dwelling Types in Toronto in 2001', 
    '2001', 
    'Dwelling Type Units', 
    'DodgerBlue'
)

bar_chart_2006 = create_bar_chart(
    dwelling_types_per_year.loc[2006], 
    'Dwelling Types in Toronto in 2006', 
    '2006', 
    'Dwelling Type Units', 
    'DeepPink'
)

bar_chart_2011 = create_bar_chart(
    dwelling_types_per_year.loc[2011], 
    'Dwelling Types in Toronto in 2006', 
    '2011', 
    'Dwelling Type Units', 
    'LimeGreen'
)

bar_chart_2016 = create_bar_chart(
    dwelling_types_per_year.loc[2016], 
    'Dwelling Types in Toronto in 2016', 
    '2016', 
    'Dwelling Type Units', 
    'DarkViolet'
)

# Create the Yearly Market Analysis view
yearly_market_analysis_column = pn.Column(
    pn.Row(bar_chart_2001, bar_chart_2006),
    pn.Row(bar_chart_2011, bar_chart_2016),
)

### Create the Shelter Costs vs. House Value view

In [115]:
average_monthly_shelter_costs = average_monthly_shelter_costs(to_data)

shelter_costs_owned_chart = create_line_chart(
    average_monthly_shelter_costs[['shelter_costs_owned']], 
    'Average Monthly Shelter Cost for Owned Dwellings in Toronto', 
    'Year', 
    'average. Monthly Shelter Cost', 
    'DodgerBlue'
)

# Line chart for rented dwellings
shelter_costs_rented_chart = create_line_chart(
    average_monthly_shelter_costs[['shelter_costs_rented']], 
    'Average Monthly Shelter Cost for Rented Dwellings in Toronto', 
    'Year', 
    'average. Monthly Shelter Cost', 
    'DeepPink'
)

average_house_value_per_year = average_house_value(to_data)

average_house_value_chart = create_line_chart(
    average_house_value_per_year, 
    'Average House Value in Toronto', 
    'Year', 
    'Avg. House Value', 
    'DodgerBlue'
)

shelter_costs_column = pn.Column(
    shelter_costs_owned_chart,
    shelter_costs_rented_chart,
    average_house_value_chart
)

### Create the Neighbourhood Analysis view

In [116]:
# average_house_values = average_house_value_snapshot(to_data)

# average_house_values_chart = create_bar_chart_row_facet(
#     average_house_values,
#     "Average House Values in Toronto per Neighbourhood",
#     'Neighbourhood', 
#     'Avg. House Value', 
#     'Avg. House Value',
# )

# neighbourhood_analysis_column = pn.Column(
#     shelter_costs_owned_chart,
#     shelter_costs_rented_chart,
#     average_house_value_chart
# )

### Group views into tabs

In [117]:
tabs = pn.Tabs(
    ("Welcome", welcome_column),
    ("Yearly Market Analysis", yearly_market_analysis_column),
    # ("Shelter Costs vs. House Value", shelter_costs_column),
    # ("Neighbourhood Analysis", geo_column),
)

# Create a tab layout for the dashboard
dashboard = pn.Column(
    "## Toronto Real Estate Analysis from 2001 to 2016",
    tabs
)

    # ("Top Expensive Neighbourhoods", geo_column),

## Serve the Panel Dashboard

In [118]:
dashboard.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'7cebb30d-0a7e-42e2-b17d-741efb313647': {'defs': …

Column
    [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Markdown(str)
            [1] Plotly(Figure)
        [1] Column
            [0] Markdown(str)
            [1] Row
                [0] HoloViews(Bars)
                [1] HoloViews(Bars)
            [2] Row
                [0] HoloViews(Bars)
                [1] HoloViews(Bars)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [119]:
# neighbourhood_map(to_data).show()

In [120]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [121]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [122]:
# average_house_value()

In [123]:
# average_value_by_neighbourhood()

In [124]:
# number_dwelling_types()

In [125]:
# average_house_value_snapshot()

In [126]:
# top_most_expensive_neighbourhoods()

In [127]:
# sunburts_cost_analysis()